In [ ]:
import os
import cv2
from imutils import paths
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import numpy as np

In [ ]:
from LabelsGenerator import generateLabel

In [2]:
import pickle

In [ ]:
imagePaths = list(sorted(paths.list_images("/home/divyanshu/Documents/dogCNN/dogDataset/Images")))
annPaths = list(sorted(os.listdir("/home/divyanshu/Documents/dogCNN/dogDataset/Annotation")))
path = "/home/divyanshu/Documents/dogCNN/dogDataset/Annotation/"
annPaths = [path + annPaths[i] for i in range(len(annPaths))]

In [ ]:
print(imagePaths[0])

In [ ]:
N = 19

In [ ]:
len(annPaths) == len(imagePaths)

In [ ]:
labels = []
for x in range(len(imagePaths)):
    img_label = []
    for i in range(N):
        for j in range(N):
            try:
                curr_label = generateLabel(imagePaths[x], 480, N, annPaths[x], i, j)
                img_label.append(curr_label)
            except Exception:
                continue
            labels.append(img_label)
    print(x)
print(labels.shape)  
pickle.dump(labels, open("labels.dat", 'wb'))

In [5]:
labels = pickle.load(open("labels.dat", 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'labels.dat'

In [ ]:
labelsarr = np.array(labels)

In [ ]:
features = []
for i in np.arange(0, len(imagePaths), 32):
    batchPaths = imagePaths[i: i + 32]
    batchImages = []
    for (j, imagePath) in enumerate(batchPaths):
        image = load_img(imagePath, target_size = (224, 224))
        image = img_to_array(image)
        image = np.expand_dims(image, axis = 0)
        batchImages.append(image)
    batchImages = np.vstack(batchImages)
    features.append(batchImages)

In [ ]:
import pickle
pickle.dump(features, open("images.bin", 'wb'))

In [ ]:
print(len(features))

In [ ]:
print(np.shape(features[0]))